In [1]:
import math as mt
import random

In [2]:
## length 자릿수의 소수 2개를 무작위로 생성하는 함수
def prime_generator(length):

    num        = 3
    gen_primes = []
    
    while True:
        is_prime = True
        len_str  = len(str(num))
        
        for idx in range(2, int(mt.sqrt(num) + 1)):

            if num % idx == 0:
                is_prime = False
                break

        ## length 자릿수인 소수들 저장
        if (is_prime) and (len_str == length): 

            ## True가 나올 확률과 False가 나올 확률을 극단적으로
            ## 불균형하게 맞춰서 추출되는 소수의 무작위성 추가

            ## 숫자 다 넣고 마지막에 2개만 뽑는 것보다는 무지성 돌다가
            ## 리스트 안에 소수 2개가 들어가면 반환하는게 속도가 더 빨라서 이 방법 채용
            bool_ = random.choices([True, False], weights = [0.1, 0.9])
            if bool_[0]: gen_primes.append(num)
                
            if len(gen_primes) == 2: return gen_primes

        ## length 자릿수를 넘어가면 멈춤
        if len_str >= (length + 1): break

        num += 1

## pi_n과 서로소인 공개키 e를 구하는 함수
def get_pub_key(n):

    pub_keys = []
    for idx in range(2, n):
        if mt.gcd(n, idx) == 1: return idx


## 확장 유클리드 호제법
## 정수론 이론을 오랜만에 봐서 이해하는데 너무 어려웠다..
def extended_euclidean_algorithm(n, m):

    s0, s1 = 1, 0
    t0, t1 = 0, 1
    
    while m != 0:

        q, n, m = n // m, m, n % m
        s0, s1  = s1, s0 - q * s1
        t0, t1  = t1, t0 - q * t1

    return s0, t0

## 개인키 e의 modulo pi_n의 역원인 개인키 d를 구하는 함수
## gcd(pub_key, pi_n) = 1 -> pub_key * s + pi_n * t = 1 일때,
## s가 pub_key (mod pi_n)의 역원임을 이용한 방법
def get_priv_key(pub_key, pi_n):

    priv_key, _ = extended_euclidean_algorithm(pub_key, pi_n)

    ## priv_key가 음수인 경우 pi_n을 더해 양수로 변환
    ## priv_key = m (mod pi_n) => priv_key + pi_n = m (mod pi_n)
    ## priv_key + pi_n (mod pi_n) => priv_key (mod pi_n) + pi_n (mod pi_n)임을 이용 

    ## e.g.) priv_key : 4, pi_n = 5 일 때,
    ## 4 = 4 (mod 5), 4 + 5 = 9 = 4 (mod 5)
    if priv_key < 0: priv_key += pi_n
    return priv_key


## 암호화와 복호화 역할을 하는 유틸 함수
def enc_n_dec(message, key, n, mode = 'enc'):

    ## 암호화의 경우 메시지를 아스키 코드로, 복호화의 경우 문자열 처리된 아스키 코드를 정수형으로 변환
    message_method = ord if mode == 'enc' else int

    ## 암호화의 경우 아스키 코드로 변환 후 암호화 된 메시지가 저장된 리스트의 join을 위해 문자열로,
    ## 복호화의 경우 복호화된 아스키 코드를 알파벳으로 변환
    convert_method = str if mode == 'enc' else chr
    join_method    = ' ' if mode == 'enc' else ''
    results = []

    for msg in message:
        result = get_modulo(message_method(msg), key, n)
        results.append(convert_method(result))

    return join_method.join(results)
        

## 숫자 RSA 암호화, 복호화 함수
def RSA(message, p_length = 3):

    print('[INFO] generate primes')
    p, q = prime_generator(length = p_length)
    n    = p * q
    pi_n = (p - 1)*(q - 1)
    
    print('[INFO] generate public key')
    pub_key   = get_pub_key(pi_n)

    print('[INFO] generate private key')
    priv_key  = get_priv_key(pub_key, pi_n)

    print('[INFO] encrypt message')
    encrypted = get_modulo(message  , pub_key , n)

    print('[INFO] decrypt message\n')
    decrypted = get_modulo(encrypted, priv_key, n)

    print(f'평문  :  {message}')
    print(f'암호문 : {encrypted}')
    print(f'복호문 : {decrypted}')


## 문자열 RSA 암호화, 복호화 함수
## 3자리 소수가 가장 빠르고, 안정성 있어서 p_length = 3으로 설정
#! 4자리 소수는 계산되는데 너무 느리고, 2자리 소수는 소수 2개가 안 뽑히는 경우도 있음.
def RSA_string(message, p_length = 3):
    
    print('[INFO] generate primes')
    p, q = prime_generator(length = p_length)
    n    = p * q
    pi_n = (p - 1)*(q - 1)
    
    print('[INFO] generate public key')
    pub_key  = get_pub_key(pi_n)
    
    print('[INFO] generate private key')
    priv_key = get_priv_key(pub_key, pi_n)

    print('[INFO] encrypt message')
    encrypted = enc_n_dec(message, pub_key, n)

    print('[INFO] decrypt message\n')
    decrypted = enc_n_dec(encrypted.split(), priv_key, n, mode = 'dec')
    
    print(f'[평문]\n {message}\n')
    print(f'[암호문]\n {encrypted}\n')
    print(f'[복호문]\n {decrypted}\n')
        

get_modulo = lambda message, key, n: message ** key % n

In [3]:
RSA(9999, p_length = 3)

[INFO] generate primes
[INFO] generate public key
[INFO] generate private key
[INFO] encrypt message
[INFO] decrypt message

평문  :  9999
암호문 : 24807
복호문 : 9999


In [4]:
RSA_string('''Freude, schoener, Goetterfunken, Tochter aus Elysium,
wir betreten, feuertrunken Himmlische dein Heiligtum,
deine Zauber binden wieder, was die Mode streng geteilt,
alle Menschen werden Brueder, so dein sanfter Fluegelt weilt.''')

[INFO] generate primes
[INFO] generate public key
[INFO] generate private key
[INFO] encrypt message
[INFO] decrypt message

[평문]
 Freude, schoener, Goetterfunken, Tochter aus Elysium,
wir betreten, feuertrunken Himmlische dein Heiligtum,
deine Zauber binden wieder, was die Mode streng geteilt,
alle Menschen werden Brueder, so dein sanfter Fluegelt weilt.

[암호문]
 796 12628 998 4902 7173 998 4057 1667 3898 418 8653 3619 998 8258 998 12628 4057 1667 6264 3619 998 493 493 998 12628 12359 4902 8258 3702 998 8258 4057 1667 5995 3619 418 8653 493 998 12628 1667 3863 4902 3898 1667 1584 8275 12290 3898 10950 4902 1081 4057 8433 12190 10950 12628 1667 1510 998 493 12628 998 493 998 8258 4057 1667 12359 998 4902 998 12628 493 12628 4902 8258 3702 998 8258 1667 2274 10950 1081 1081 8275 10950 3898 418 8653 998 1667 7173 998 10950 8258 1667 2274 998 10950 8275 10950 5356 493 4902 1081 4057 8433 7173 998 10950 8258 998 1667 5790 3863 4902 1510 998 12628 1667 1510 10950 8258 7173 998 8258 1667 1219